In [1]:
'''
  code by Tae Hwan Jung(Jeff Jung) @graykode, modify by wmathor
'''
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

dtype = torch.FloatTensor

In [2]:
char_arr = [c for c in 'abcdefghijklmnopqrstuvwxyz'] # ['a', 'b', 'c',...]
word2idx = {n: i for i, n in enumerate(char_arr)}
idx2word = {i: w for i, w in enumerate(char_arr)}
n_class = len(word2idx) # number of class(=number of vocab)

seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']

# TextLSTM Parameters
n_step = len(seq_data[0]) - 1 # (=3) 就是历史看三个字母，预测最后一个字母
n_hidden = 128

def make_data(seq_data):
    input_batch, target_batch = [], []

    for seq in seq_data:
        input = [word2idx[n] for n in seq[:-1]] # 'm', 'a' , 'k' is input
        target = word2idx[seq[-1]] # 'e' is target
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return torch.Tensor(input_batch), torch.LongTensor(target_batch)

input_batch, target_batch = make_data(seq_data)
dataset = Data.TensorDataset(input_batch, target_batch)
loader = Data.DataLoader(dataset, 3, True)

In [3]:
class TextLSTM(nn.Module):
    def __init__(self):
        super(TextLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden)
        # fc
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, X):
        # X: [batch_size, n_step, n_class]
        batch_size = X.shape[0]
        input = X.transpose(0, 1)  # X : [n_step, batch_size, n_class]

        hidden_state = torch.zeros(1, batch_size, n_hidden)   #是初始化的h0,c0 [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        cell_state = torch.zeros(1, batch_size, n_hidden)    # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]

        outputs, (_, _) = self.lstm(input, (hidden_state, cell_state))
        outputs = outputs[-1]  # [batch_size, n_hidden]
        model = self.fc(outputs)
        return model


model = TextLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [4]:
# Training
for epoch in range(1000):
  for x, y in loader:
    pred = model(x)
    loss = criterion(pred, y)
    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 0100 cost = 0.021760
Epoch: 0100 cost = 0.214454
Epoch: 0100 cost = 0.016971
Epoch: 0100 cost = 0.040900
Epoch: 0200 cost = 0.003466
Epoch: 0200 cost = 0.041558
Epoch: 0200 cost = 0.002600
Epoch: 0200 cost = 0.002137
Epoch: 0300 cost = 0.007363
Epoch: 0300 cost = 0.000711
Epoch: 0300 cost = 0.008049
Epoch: 0300 cost = 0.000365
Epoch: 0400 cost = 0.003190
Epoch: 0400 cost = 0.003718
Epoch: 0400 cost = 0.000906
Epoch: 0400 cost = 0.000536
Epoch: 0500 cost = 0.000352
Epoch: 0500 cost = 0.001526
Epoch: 0500 cost = 0.000329
Epoch: 0500 cost = 0.008434
Epoch: 0600 cost = 0.001356
Epoch: 0600 cost = 0.000167
Epoch: 0600 cost = 0.000261
Epoch: 0600 cost = 0.003648
Epoch: 0700 cost = 0.000073
Epoch: 0700 cost = 0.000188
Epoch: 0700 cost = 0.000854
Epoch: 0700 cost = 0.002573
Epoch: 0800 cost = 0.000152
Epoch: 0800 cost = 0.000144
Epoch: 0800 cost = 0.000526
Epoch: 0800 cost = 0.001806
Epoch: 0900 cost = 0.000379
Epoch: 0900 cost = 0.000104
Epoch: 0900 cost = 0.000546
Epoch: 0900 cost = 0

In [5]:
inputs = [sen[:3] for sen in seq_data]
predict = model(input_batch).data.max(1, keepdim=True)[1]
print(inputs, '->', [idx2word[n.item()] for n in predict.squeeze()])

['mak', 'nee', 'coa', 'wor', 'lov', 'hat', 'liv', 'hom', 'has', 'sta'] -> ['e', 'd', 'l', 'd', 'e', 'e', 'e', 'e', 'h', 'r']
